In [418]:
import pandas as pd
import re
import fasttext as ft
import translators as ts
ft_model = ft.load_model("lid.176.bin")
from nrclex import NRCLex
import spacy 
import en_core_web_lg
nlp = en_core_web_lg.load()
import nltk
nltk.download('punkt')
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

[nltk_data] Downloading package punkt to /home/kwaks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [350]:
from tqdm.notebook import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoderLabse = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

def normalization(embeddings):
    norms = np.linalg.norm(embeddings, 2, axis=1, keepdims=True)
    return embeddings/norms

def val_ct_dict(obj):
    unique, counts = np.unique(obj, return_counts=True)
    val_ct = dict(zip(unique, counts))
    val_ct = dict(reversed(sorted(val_ct.items(), key=lambda item: item[1])))
    return val_ct

def embedd(sentence):
    if isinstance(sentence, str):
        embedding = tf.constant([sentence])
    else:
        embedding = tf.constant(sentence)
    embedding = encoderLabse(preprocessor(embedding))["default"]
    embedding = normalization(embedding)
    return embedding

def embedd_list(list_obj):
    embedded = np.zeros((len(list_obj), 768))
    with tqdm(total=len(list_obj)) as pbar:
        for i in range(len(list_obj)):
            embedded[i] = embedd(list_obj[i])
            pbar.update(1)
    return embedded

def val_ct_dict(obj):
    unique, counts = np.unique(obj, return_counts=True)
    val_ct = dict(zip(unique, counts))
    val_ct = dict(reversed(sorted(val_ct.items(), key=lambda item: item[1])))
    return val_ct


2022-07-11 00:37:22.693370: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-11 00:37:22.693387: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-11 00:37:22.693398: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Kwakuter): /proc/driver/nvidia/version does not exist
2022-07-11 00:37:22.693549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
tweets = pd.ExcelFile('tweets_sample.xlsx')
langs = tweets.sheet_names 

In [205]:

def get_emotional(text):
    emotion = NRCLex(text)
    emo = emotion.top_emotions
    if not sum(dict(emo).values()):
        return ("neutral")
    else:
        return emo
    
def process(text,retweet_name):
    text = re.sub(r'http\S+', '', text)
    if isinstance(retweet_name, str) and len(retweet_name):
        text = re.sub(r'@\w+', retweet_name.replace(" ", "_"), text)
    else:
        text = re.sub(r'@', "", text)
    sent = text.split(".")
    for s in sent:
        if len(s)>1:
            print("-----")
            l = ft_model.predict(s)[0][0][-2:]
            print(l)
            if l != 'en':
                s = ts.google(s, from_language=l, to_language='en')

            emo = get_emotional(s)
            ents = [t for t in nlp(s).ents]
            print(s,ents,emo)

    print("\n____________________________________________________________________________________\n")   
    words = text
    return pd.Series([text, words])


In [461]:
words = {}
l_ = ["UK.2"]
from spacy.lang.en import STOP_WORDS
prefix_re = re.compile('''^\$[a-zA-Z0-9]''')
def custom_tokenizer(nlp):
        return spacy.tokenizer.Tokenizer(nlp.vocab, prefix_search=prefix_re.search)
nlp.tokenizer = custom_tokenizer(nlp) 

def clear_lemma(text):
    text = re.sub(r'@', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = text.lower()
    text = re.sub(r'\n ', '', text)
    text = re.sub(r'@', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'\.|\,|\:', ' ', text)
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = re.sub(r'prime minister', 'prime_minister', text)
    text = re.sub(r'pm', 'prime_minister', text)
    text = re.sub(r'united kingdom', 'united kingdom', text)
    text_lemmatized = [t.lemma_ for t in nlp(text) if t.lemma_ not in STOP_WORDS and 
                       not re.match(r'\s',t.lemma_)
                      and len(t.lemma_)>1]
    return text_lemmatized 

def bi_tri_grams(text):
    return pd.Series([list(nltk.bigrams(text)), list(nltk.trigrams(text))])

for lang in l_:
    corpus = tweets.parse(lang)
    corpus.text = corpus.text.apply(lambda x: clear_lemma(x))
    corpus[["bi","tri"]] = corpus.text.apply(lambda x: bi_tri_grams(x))


In [462]:
bi = np.array([" ".join(sorted(list(x))) for xs in corpus.bi.values for x in xs])
bi_u = [i.split(" ") for i in list(val_ct_dict(bi).keys())]
bi_u = list(zip(*bi_u))
bigrams = pd.DataFrame(zip(bi_u[0],bi_u[1],val_ct_dict(bi).values()),columns=["bi1","bi2","cnt"])
bigrams = bigrams[bigrams["cnt"]>=3]

In [463]:
words_from_bigram = pd.unique(bigrams[["bi2","bi1"]].values.ravel('K'))

In [465]:
# embeddings_bi = [bigrams.bi1.apply(embedd),bigrams.bi2.apply(embedd)]

In [466]:
#t-SNE
words_bi_embedded = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(embedd(words_from_bigram))
#PCA
# words_bi_embedded = PCA(n_components=2).fit_transform(embedd(words_from_bigram))



In [467]:
words_bi_dict = dict(zip(words_from_bigram, words_bi_embedded))

In [468]:
plot_df = pd.DataFrame(words_bi_embedded,columns=["x","y"])
plot_df["words"] = words_from_bigram


In [469]:


# import pandas as pd
import numpy as np
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from bokeh.plotting import figure, output_file, output_notebook, show , save
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, ColumnDataSource, Legend, LabelSet

output_file(filename="graf_bigramow.html", title="Graf bigramow")


timespan = range(2002,2021)
hover = HoverTool()
hover.tooltips = [
("word", "@words"),
# ("gęstość zaludnienia", "@density osób/km2"),
# ("powierzchnia gminy zajeta przez park %", "@park_percent{0.2f} %"),
# ("dochód na mieszkańca", "@y{int} zł"),
# ("rok", "@x"),
]
hover.formatters={'datetime': 'datetime'}


zoom = BoxZoomTool()
reset = ResetTool()

graph = figure(title = "bigrams t-sne",
               plot_width=1100, plot_height=900,
#                tools=[hover]
              ) 
#     graph.xaxis.axis_label = 'Rok'
#     graph.yaxis.axis_label = 'Dochód [zł / osobę]'

data = ColumnDataSource({"x": plot_df.x.values, "y": plot_df.y.values, 
                         "words": list(plot_df.words.values),

                        })


for bi1,bi2,cnt in zip(bigrams.bi1.values,bigrams.bi2.values,bigrams.cnt.values):
    graph.line([words_bi_dict[bi1][0],words_bi_dict[bi2][0]], 
           [words_bi_dict[bi1][1],words_bi_dict[bi2][1]], 
                  color = "red", 
                  line_width=cnt*0.2,
                  )
graph.circle("x", "y", source=data, size=10,color="navy", alpha=1)

labels = LabelSet(x='x', y='y', text='words',
         x_offset=5, y_offset=5, source=data)
graph.add_layout(labels)
# graph.add_layout(graph.legend[0], 'right')
graph.legend.click_policy="hide"
save(graph)




/tmp/ipykernel_283815/1610955349.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/home/kwaks/gejopandas/lib/python3.10/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


'/home/kwaks/Desktop/ukraina/graf_bigramow.html'

In [435]:
bigrams

,bi1,bi2,cnt
0,support,ukraine,28
1,invasion,ukraine,22
2,people,ukraine,14
3,president,zelenskyyua,13
4,borisjohnson,prime_minister,13
...,...,...,...
97,arrive,uk,3
98,application,visa,3
99,ally,work,3
100,ally,nato,3
